In [1]:
import pandas as pd
import nltk.stem as ns
import json
from pickle import dump, load
from nltk.corpus import stopwords
import nltk
import re
from collections import Counter

rest = pd.read_csv('/afs/cs.wisc.edu/u/l/e/leng/private/STAT628-M3/Philadelphia_Veg_data.csv')
rest.head(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,bTve2mwLk5Zc01vRKqc2KQ,Red Hook Coffee & Tea,765 S 4th St,Philadelphia,PA,19147,39.938552,-75.149636,4.5,183,1,"{'BikeParking': 'True', 'OutdoorSeating': 'Tru...","Restaurants, Breakfast & Brunch, Vegan, Bagels...","{'Monday': '7:0-18:0', 'Tuesday': '7:0-18:0', ..."
1,JomDoQafikclSVVCPkWjnA,Cedar Park Cafe,4914 Baltimore Ave,Philadelphia,PA,19143,39.947960,-75.221980,4.0,301,1,"{'WiFi': ""u'no'"", 'OutdoorSeating': 'False', '...","Diners, Restaurants, Sandwiches, Breakfast & B...","{'Monday': '7:0-15:0', 'Tuesday': '7:0-15:0', ..."
2,IgNDlyLERBwFGphgcC5VDQ,Cosi,1128 Walnut St,Philadelphia,PA,19107,39.948751,-75.160449,2.5,63,0,"{'OutdoorSeating': 'True', 'RestaurantsPriceRa...","Sandwiches, Vegetarian, Pizza, Food, Coffee & ...","{'Monday': '6:0-22:0', 'Tuesday': '6:0-22:0', ..."


In [2]:
rest = rest[rest['review_count'] > 10]
rest.shape[0]

245

In [5]:
rest_id = rest['business_id'].tolist()

reviews = []

count = 0

with open('/afs/cs.wisc.edu/u/l/e/leng/public/yelp_Fall2023/review.json') as f: #, encoding='utf-8'
    while True:
        count += 1
        line = f.readline()
        if not line: # 到 EOF，返回空字符串，则终止循环
            break
        review = json.loads(line)
        if review['business_id'] in rest_id:
            reviews.append(review)
        if count%100000 == 0:
            print(count, end = ' ')

In [10]:
reviews[0]

{'review_id': '3dVcGYz6GokuEytLrfG8bA',
 'user_id': 'FEI0XkOrUHufSW_rfOTPAA',
 'business_id': 'Dv6RfXLYe1atjgz3Xf4GGw',
 'stars': 4.0,
 'useful': 1,
 'funny': 0,
 'cool': 0,
 'text': 'Good food, reasonably priced, and nice staff. Not ideal for eating in, but since they are going for a fast food approach to vegan food, that makes a lot of sense. I know the buffalo bella is the dish that they push the hardest, but for my money the Cheesesteak is the way to go. And pro-tip: Get the sweet potato fries.',
 'date': '2012-12-04 23:58:36'}

In [ ]:
review_id = [r['review_id'] for r in reviews]
user_id = [r['user_id'] for r in reviews]
business_id = [r['business_id'] for r in reviews]
stars = [r['stars'] for r in reviews]
useful = [r['useful'] for r in reviews]
funny = [r['funny'] for r in reviews]
cool = [r['cool'] for r in reviews]
text = [r['text'] for r in reviews]
date = [r['date'] for r in reviews]

tmp_dic = {'review_id': review_id, 'business_id': business_id, 'stars': stars, 'text': text, 'date': date, 'user_id': user_id,
           'useful': useful, 'funny': funny, 'cool': cool}

vegan_review = pd.DataFrame(tmp_dic)

In [3]:
vegan_review = load(open('/afs/cs.wisc.edu/u/l/e/leng/private/STAT628-M3/vegan_review', 'rb'))
vegan_review

,review_id,business_id,stars,text,date,user_id,useful,funny,cool
0,3dVcGYz6GokuEytLrfG8bA,Dv6RfXLYe1atjgz3Xf4GGw,4.0,"Good food, reasonably priced, and nice staff. ...",2012-12-04 23:58:36,FEI0XkOrUHufSW_rfOTPAA,1,0,0
1,2fD2Do7xBjoahST3FmiwcQ,AY1SyySRyTnBwBdf7qfGxQ,5.0,"From the second you walk in the door, you're g...",2009-08-04 18:04:15,avQwKRvXc21koLCbahFEig,3,0,2
2,nCdhMSQA0apDuB_oho5ang,Dv6RfXLYe1atjgz3Xf4GGw,4.0,Super lunch option\nWatch out for the yoga mat...,2012-08-15 19:17:49,2dyfZNhtyuHdestczTgWjQ,0,0,2
3,wUHUieO_gckt0HcrrI63PQ,Dv6RfXLYe1atjgz3Xf4GGw,5.0,"Wow. I am not a vegetarian, nor am I a health-...",2013-02-22 00:48:28,5ucOKo929N6dPfNjLuFoEA,1,0,0
4,yyAycjhj9pBUWYlZd75IZg,bTve2mwLk5Zc01vRKqc2KQ,5.0,This might be my favorite coffee shop in the c...,2016-05-07 01:15:03,rZ4rHwgqTnP4NUcpG_MQvw,0,0,0
...,...,...,...,...,...,...,...,...,...
48226,KsCTNz-iA_2ymuxrcIg-ig,2CDI713ATuxHfnB5b-sBdw,5.0,I am a DIE HARD omnivore! I see no reason to g...,2020-08-25 00:01:58,JeL6lnjXSsVs1FiSYH6ghA,5,1,1
48227,V93jD0hG6A1FC4MGfoyp1Q,2CDI713ATuxHfnB5b-sBdw,4.0,This is an amazing place that has so much to o...,2014-10-21 18:37:54,Sd357RRT7uLQ3v52aQaLqw,3,0,2
48228,dQq5xvEBTo34vnokDWSFwg,4D_GsaRoqk7P09VOcgezlQ,3.0,"Overpriced, small portions.\n\nI ordered a bag...",2017-07-10 14:56:30,tQ8N3-j2Ckc_6ajPRg01tw,3,0,0
48229,HDujUiWKWgYKurpKhlSTzQ,4D_GsaRoqk7P09VOcgezlQ,3.0,It's a well established fact that McThrill is ...,2009-08-11 04:27:55,bJ5FtCtZX3ZZacz2_2PJjA,4,0,2


In [4]:
vegan_review = vegan_review[vegan_review['business_id'].isin(rest['business_id'].tolist())]
vegan_review.shape[0]

47991

In [5]:
stop = stopwords.words('english')
stop += ['Im', 'Ive', 'didnt', 'cant', 'Id', 'Ill', 'do', 'youre', 'youd', 'youll']

texts = vegan_review['text'].tolist()
filter_text = re.sub('[{}]'.format('!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~'),'',texts[0]).replace('\n', '')
filter_text = [word for word in filter_text.split(' ') if word not in stop]
print(texts[0])
print(filter_text)

t = [nltk.pos_tag([w])[0] for w in filter_text]
print(t)

Good food, reasonably priced, and nice staff. Not ideal for eating in, but since they are going for a fast food approach to vegan food, that makes a lot of sense. I know the buffalo bella is the dish that they push the hardest, but for my money the Cheesesteak is the way to go. And pro-tip: Get the sweet potato fries.
['Good', 'food', 'reasonably', 'priced', 'nice', 'staff', 'Not', 'ideal', 'eating', 'since', 'going', 'fast', 'food', 'approach', 'vegan', 'food', 'makes', 'lot', 'sense', 'I', 'know', 'buffalo', 'bella', 'dish', 'push', 'hardest', 'money', 'Cheesesteak', 'way', 'go', 'And', 'pro-tip', 'Get', 'sweet', 'potato', 'fries']
[('Good', 'JJ'), ('food', 'NN'), ('reasonably', 'RB'), ('priced', 'VBN'), ('nice', 'JJ'), ('staff', 'NN'), ('Not', 'RB'), ('ideal', 'NN'), ('eating', 'VBG'), ('since', 'IN'), ('going', 'VBG'), ('fast', 'NN'), ('food', 'NN'), ('approach', 'NN'), ('vegan', 'NN'), ('food', 'NN'), ('makes', 'VBZ'), ('lot', 'NN'), ('sense', 'NN'), ('I', 'PRP'), ('know', 'VB'), 

In [6]:
verb = ['VBD', 'VB', 'VBG', 'VBN', 'VBZ', 'VBG']
noun = ['NN', 'NNS', 'NNP', 'NNPS']
adj = ['JJ', 'JJR', 'JJS']
adv = ['RB', 'RBR', 'RBS', 'UH']

v_ = [t[i][0].lower() for i in range(len(t)) if t[i][1] in verb]
n_ = [t[i][0] for i in range(len(t)) if t[i][1] in noun]
adj_ =  [t[i][0].lower() for i in range(len(t)) if t[i][1] in adj]
adv_ =  [t[i][0].lower() for i in range(len(t)) if t[i][1] in adv]

In [7]:
lemmatizer = ns.WordNetLemmatizer()
 
v_ = [lemmatizer.lemmatize(v, pos='v') for v in v_]
n_ = [lemmatizer.lemmatize(n, pos='n') for n in n_]

In [8]:
print(v_)
print(n_)
print(adj_)
print(adv_)

['price', 'eat', 'go', 'make', 'know', 'go', 'get']
['food', 'staff', 'ideal', 'fast', 'food', 'approach', 'vegan', 'food', 'lot', 'sense', 'buffalo', 'bella', 'dish', 'push', 'hardest', 'money', 'Cheesesteak', 'way', 'pro-tip', 'sweet', 'potato', 'fry']
['good', 'nice']
['reasonably', 'not']


In [10]:
verbs = []
nouns = []
adjs = []
advs = []

count = 0

for t in texts:
    count += 1
    filter_text = re.sub('[{}]'.format('!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~'),'',t).replace('\n', '')
    filter_text = [word for word in filter_text.split(' ') if word not in stop]
    
    tag = [nltk.pos_tag([w])[0] for w in filter_text]
    
    v_ = [tag[i][0].lower() for i in range(len(tag)) if tag[i][1] in verb]
    n_ = [tag[i][0] for i in range(len(tag)) if tag[i][1] in noun]
    adj_ =  [tag[i][0].lower() for i in range(len(tag)) if tag[i][1] in adj]
    adv_ =  [tag[i][0].lower() for i in range(len(tag)) if tag[i][1] in adv]
    
    v_ = [lemmatizer.lemmatize(v, pos='v') for v in v_]
    n_ = [lemmatizer.lemmatize(n, pos='n') for n in n_]
    
    verbs.append(v_)
    nouns.append(n_)
    adjs.append(adj_)
    advs.append(adv_)
    
    if count%1000 == 0:
        print(count, end = ' ')

1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 45000 46000 47000 

In [11]:
vegan_word = vegan_review[['business_id', 'text', 'stars']]
vegan_word['adj'] = adjs
vegan_word['adv'] = advs
vegan_word['noun'] = nouns
vegan_word['verb'] = verbs

vegan_word

/tmp/ipykernel_508839/2050911339.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vegan_word['adj'] = adjs
/tmp/ipykernel_508839/2050911339.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vegan_word['adv'] = advs
/tmp/ipykernel_508839/2050911339.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

,business_id,text,stars,adj,adv,noun,verb
0,Dv6RfXLYe1atjgz3Xf4GGw,"Good food, reasonably priced, and nice staff. ...",4.0,"[good, nice]","[reasonably, not]","[food, staff, ideal, fast, food, approach, veg...","[price, eat, go, make, know, go, get]"
1,AY1SyySRyTnBwBdf7qfGxQ,"From the second you walk in the door, you're g...",5.0,"[second, adorable, lucky, recent, vegetarian, ...","[particularly, actually, also, always, really,...","[walk, door, woman, counter, week, shell, name...","[greet, seem, come, know, serve, go, enjoy, pr..."
2,Dv6RfXLYe1atjgz3Xf4GGw,Super lunch option\nWatch out for the yoga mat...,4.0,"[last, great, green, delicious, great, busy, b...","[maybe, yet, especially, also, also, friendly,...","[Super, lunch, optionWatch, yoga, matsYou, won...","[open, amaze, pair, get, say, get, come, take,..."
3,Dv6RfXLYe1atjgz3Xf4GGw,"Wow. I am not a vegetarian, nor am I a health-...",5.0,"[vegetarian, delicious, good]",[well],"[Wow, health-food, nut, random, diet, binge, p...","[talk, try, go, get, put, create, go, greet, w..."
4,bTve2mwLk5Zc01vRKqc2KQ,This might be my favorite coffee shop in the c...,5.0,"[unbelievable, fresh, delicious, enjoyable, gr...","[definitely, especially, well]","[favorite, coffee, shop, city, food, Anything,...","[order, seat, check]"
...,...,...,...,...,...,...,...
48226,2CDI713ATuxHfnB5b-sBdw,I am a DIE HARD omnivore! I see no reason to g...,5.0,"[hard, hard, personal, uber-picky, seven-year-...","[so, philly, then, so, finally, long, almost, ...","[DIE, omnivore, reason, way, seek, spot, doesn...","[see, go, talk, wonder, get, become, get, visi..."
48227,2CDI713ATuxHfnB5b-sBdw,This is an amazing place that has so much to o...,4.0,"[much, vegetarian, many, great, casual, rendez...","[well, amazingly, nicely, thoroughly, even, st...","[place, offer, vegan, omnivore, carnivore, enj...","[amaze, amaze, design, say, let, consult, know..."
48228,4D_GsaRoqk7P09VOcgezlQ,"Overpriced, small portions.\n\nI ordered a bag...",3.0,"[small, good, small, small, good]","[however, hardly]","[portionsI, bagel, sandwich, thin, slice, toma...","[overprice, order, taste, top, taste, overpric..."
48229,4D_GsaRoqk7P09VOcgezlQ,It's a well established fact that McThrill is ...,3.0,"[interested, much, much, different, strong, de...","[well, already, so, anyway, aside, nside, norm...","[fact, McThrill, hooked, indie, coffeehouse, k...","[establish, frequent, try, square, decide, giv..."


In [12]:
with open('/afs/cs.wisc.edu/u/l/e/leng/private/STAT628-M3/vegan_word','wb') as filepath: 
    dump(vegan_word, filepath)

In [44]:
def invalid_words(words):
    l = []
    for w in words:
        l += w

    c = Counter(l)
    freq = list(c.values())
    n_invalid = sum([f<=10 for f in freq])
    invalid = c.most_common()[-n_invalid:-1]
    invalid = [i_n[0] for i_n in invalid]
    invalid.append('')
    
    return invalid

In [46]:
invalid_noun = invalid_words(nouns)
len(invalid_noun)

70921

In [49]:
clean_nouns = []
count = 0
for n in nouns:
    count += 1
    clean_n = list(set(n)-set(invalid_noun))
    clean_nouns.append(clean_n)
    
    if count%1000 == 0:
        print(count, end = ' ')

1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 45000 46000 47000 

In [50]:
def clean_words(invalid, words):
    clean_words = []
    
    count = 0
    for w in words:
        count += 1
        clean_w = list(set(w)-set(invalid))
        clean_words.append(clean_w)

        if count%1000 == 0:
            print(count, end = ' ')
            
    return clean_words

In [51]:
invalid_verb = invalid_words(verbs)
print(len(invalid_verb))

clean_verbs = clean_words(invalid_verb, verbs)

4308
1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 45000 46000 47000 

In [52]:
invalid_adj = invalid_words(adjs)
print(len(invalid_adj))

clean_adjs = clean_words(invalid_adj, adjs)

5942
1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 45000 46000 47000 

In [53]:
invalid_adv = invalid_words(advs)
print(len(invalid_adv))

clean_advs = clean_words(invalid_adv, advs)

2839
1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 45000 46000 47000 

In [54]:
vegan_word_filtered = vegan_word[['business_id', 'text', 'stars']]

vegan_word_filtered['adj'] = clean_adjs
vegan_word_filtered['adv'] = clean_advs
vegan_word_filtered['noun'] = clean_nouns
vegan_word_filtered['verb'] = clean_verbs

vegan_word_filtered

/tmp/ipykernel_508839/4123554432.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vegan_word_filtered['adj'] = clean_adjs
/tmp/ipykernel_508839/4123554432.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vegan_word_filtered['adv'] = clean_advs
/tmp/ipykernel_508839/4123554432.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

,business_id,text,stars,adj,adv,noun,verb
0,Dv6RfXLYe1atjgz3Xf4GGw,"Good food, reasonably priced, and nice staff. ...",4.0,"[good, nice]","[reasonably, not]","[sense, vegan, lot, money, hardest, dish, fry,...","[go, get, make, eat, price, know]"
1,AY1SyySRyTnBwBdf7qfGxQ,"From the second you walk in the door, you're g...",5.0,"[delicious, adorable, incredible, addictive, v...","[back, actually, also, particularly, always, w...","[chicken, door, week, deli, everything, people...","[go, seem, greet, amaze, serve, make, mention,..."
2,Dv6RfXLYe1atjgz3Xf4GGw,Super lunch option\nWatch out for the yoga mat...,4.0,"[impossible, delicious, hard, great, green, ba...","[long, also, friendly, everywhere, nearly, may...","[someone, door, space, thing, HipCity, crispy,...","[wait, open, take, get, rectify, amaze, pair, ..."
3,Dv6RfXLYe1atjgz3Xf4GGw,"Wow. I am not a vegetarian, nor am I a health-...",5.0,"[good, delicious, vegetarian]",[well],"[front, people, something, company, salad, bin...","[go, talk, wait, create, get, try, greet, put]"
4,bTve2mwLk5Zc01vRKqc2KQ,This might be my favorite coffee shop in the c...,5.0,"[delicious, nice, great, enjoyable, unbelievab...","[definitely, especially, well]","[favorite, worth, comfy, coffee, lot, seat, co...","[order, seat, check]"
...,...,...,...,...,...,...,...
48226,2CDI713ATuxHfnB5b-sBdw,I am a DIE HARD omnivore! I see no reason to g...,5.0,"[hard, fondue, vegetarian, green, black, great...","[then, almost, forward, long, even, so, still,...","[ice, Vedge, year, heirloom, outdoor, Squash, ...","[go, become, see, love, whip, seat, try, amaze..."
48227,2CDI713ATuxHfnB5b-sBdw,This is an amazing place that has so much to o...,4.0,"[casual, vegetarian, much, great, many, outsta...","[nicely, amazingly, already, even, enough, sti...","[ice, work, vegan, melt, richness, thing, omni...","[let, design, amaze, come, say, know]"
48228,4D_GsaRoqk7P09VOcgezlQ,"Overpriced, small portions.\n\nI ordered a bag...",3.0,"[small, good]","[however, hardly]","[café, avocado, bagel, thin, cream, Plenty, bi...","[go, get, top, taste, overprice, order]"
48229,4D_GsaRoqk7P09VOcgezlQ,It's a well established fact that McThrill is ...,3.0,"[soft, grand, delectable, strong, much, inclin...","[aside, already, normally, well, apparently, a...","[n, ice, work, look, occasion, hooked, knew, s...","[decide, give, try, see, frequent, manage, est..."


In [55]:
with open('/afs/cs.wisc.edu/u/l/e/leng/private/STAT628-M3/vegan_word_filtered','wb') as filepath: 
    dump(vegan_word_filtered, filepath)

In [63]:
def word_counter(words):
    l = []
    for w in words:
        l += w

    c = Counter(l)
    n_invalid = sum([f<=10 for f in freq])
    common = c.most_common()[0:101]
    
    return common

In [64]:
common_noun = word_counter(nouns)[1:]
common_noun

[('food', 35413),
 ('place', 27969),
 ('time', 18416),
 ('menu', 12595),
 ('vegan', 12284),
 ('restaurant', 11543),
 ('service', 11315),
 ('order', 10559),
 ('dish', 10002),
 ('salad', 9343),
 ('try', 8983),
 ('chicken', 8299),
 ('love', 8219),
 ('dont', 8015),
 ('meal', 7818),
 ('friend', 7523),
 ('sauce', 7425),
 ('thing', 7320),
 ('wait', 6827),
 ('flavor', 6718),
 ('table', 6622),
 ('option', 6408),
 ('staff', 6372),
 ('drink', 6320),
 ('sandwich', 6241),
 ('eat', 6049),
 ('favorite', 5894),
 ('experience', 5884),
 ('cheese', 5792),
 ('people', 5705),
 ('taste', 5701),
 ('think', 5660),
 ('lunch', 5544),
 ('price', 5497),
 ('fry', 5390),
 ('bit', 5354),
 ('pizza', 5279),
 ('bar', 5257),
 ('day', 5202),
 ('way', 5146),
 ('sweet', 4948),
 ('recommend', 4891),
 ('night', 4834),
 ('spot', 4765),
 ('dinner', 4677),
 ('everything', 4638),
 ('side', 4631),
 ('want', 4578),
 ('brunch', 4435),
 ('lot', 4339),
 ('something', 4336),
 ('meat', 4312),
 ('star', 4290),
 ('hour', 4281),
 ('tasty'

In [65]:
common_verb = word_counter(verbs)[:-1]
common_verb

[('get', 27859),
 ('go', 23937),
 ('come', 17431),
 ('make', 15630),
 ('order', 12609),
 ('say', 9677),
 ('take', 9258),
 ('give', 7977),
 ('amaze', 7565),
 ('try', 7222),
 ('know', 5476),
 ('love', 4939),
 ('taste', 4774),
 ('look', 4739),
 ('eat', 4470),
 ('see', 4250),
 ('seat', 3990),
 ('fry', 3645),
 ('find', 3478),
 ('want', 3331),
 ('ask', 3304),
 ('serve', 3234),
 ('enjoy', 3087),
 ('wait', 3076),
 ('add', 3049),
 ('seem', 2940),
 ('din', 2795),
 ('expect', 2751),
 ('fill', 2587),
 ('decide', 2381),
 ('use', 2283),
 ('walk', 2175),
 ('grill', 1937),
 ('call', 1912),
 ('like', 1900),
 ('do', 1873),
 ('let', 1801),
 ('leave', 1657),
 ('include', 1657),
 ('keep', 1651),
 ('start', 1573),
 ('stop', 1472),
 ('visit', 1449),
 ('excite', 1421),
 ('have', 1406),
 ('work', 1396),
 ('arrive', 1385),
 ('end', 1339),
 ('season', 1335),
 ('interest', 1306),
 ('roast', 1269),
 ('remember', 1245),
 ('dress', 1228),
 ('believe', 1182),
 ('share', 1169),
 ('recommend', 1141),
 ('run', 1138),
 (

In [68]:
common_adj = word_counter(adjs)[:-1]
common_adj

[('good', 26594),
 ('great', 17729),
 ('delicious', 12939),
 ('best', 8882),
 ('little', 8173),
 ('nice', 7813),
 ('much', 7055),
 ('fresh', 6298),
 ('small', 5513),
 ('vegetarian', 4916),
 ('many', 4044),
 ('next', 3997),
 ('hot', 3737),
 ('happy', 3662),
 ('last', 3438),
 ('different', 3183),
 ('new', 3171),
 ('special', 3157),
 ('bad', 3145),
 ('full', 2899),
 ('french', 2896),
 ('healthy', 2754),
 ('big', 2745),
 ('overall', 2653),
 ('free', 2411),
 ('indian', 2373),
 ('large', 2342),
 ('disappointed', 2258),
 ('huge', 2233),
 ('whole', 2184),
 ('fantastic', 2144),
 ('hard', 2030),
 ('least', 1905),
 ('able', 1877),
 ('open', 1826),
 ('busy', 1790),
 ('high', 1761),
 ('real', 1754),
 ('green', 1707),
 ('extra', 1704),
 ('attentive', 1664),
 ('second', 1644),
 ('wrong', 1581),
 ('regular', 1444),
 ('several', 1366),
 ('red', 1334),
 ('incredible', 1331),
 ('expensive', 1312),
 ('live', 1299),
 ('old', 1256),
 ('local', 1247),
 ('prepared', 1210),
 ('soft', 1194),
 ('reasonable', 118

In [70]:
common_adv = word_counter(advs)[:-1]
common_adv

[('really', 15315),
 ('also', 13514),
 ('back', 12820),
 ('definitely', 8935),
 ('well', 8649),
 ('even', 8551),
 ('philly', 6994),
 ('pretty', 6785),
 ('always', 6766),
 ('first', 6745),
 ('friendly', 6607),
 ('never', 5522),
 ('ever', 5196),
 ('better', 5179),
 ('still', 4836),
 ('so', 4444),
 ('enough', 3595),
 ('long', 3427),
 ('atmosphere', 3167),
 ('probably', 3025),
 ('however', 2957),
 ('quite', 2703),
 ('actually', 2566),
 ('absolutely', 2542),
 ('away', 2541),
 ('not', 2505),
 ('maybe', 2470),
 ('especially', 2467),
 ('usually', 2444),
 ('highly', 2274),
 ('almost', 2246),
 ('else', 2035),
 ('very', 2033),
 ('far', 1937),
 ('perfectly', 1875),
 ('finally', 1740),
 ('extremely', 1718),
 ('yet', 1603),
 ('less', 1578),
 ('instead', 1572),
 ('often', 1512),
 ('just', 1483),
 ('close', 1432),
 ('soon', 1326),
 ('already', 1314),
 ('quickly', 1283),
 ('later', 1259),
 ('totally', 1184),
 ('honestly', 1160),
 ('ago', 1153),
 ('sometimes', 1094),
 ('rather', 1074),
 ('completely', 1

In [71]:
most_common = {'common_noun': common_noun, 'common_verb': common_verb,'common_adj': common_adj,'common_adv': common_adv}

with open('/afs/cs.wisc.edu/u/l/e/leng/private/STAT628-M3/most_common_words','wb') as filepath: 
    dump(most_common, filepath)